In [31]:
import wandb
wandb.login()

True

In [40]:
import numpy as np
import pandas as pd
import torch.nn as nn
import h5py
import pickle
import torch
import time
import yaml
import copy
import sys
import time
import os
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

In [41]:
import yaml
config_wbd = {}
with open('/home/quang/Documents/XAI_env-main/Code/config.yml', 'r') as f:
    config_wdb = yaml.safe_load(f)

In [42]:
config_wdb

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'test_loss_avg'},
 'parameters': {'batch_size': {'values': [128]},
  'd_model': {'value': 16},
  'dff': {'values': [128, 256]},
  'dropout': {'distribution': 'uniform', 'min': 0.05, 'max': 0.15},
  'l_win': {'distribution': 'int_uniform', 'min': 120, 'max': 125},
  'lr': {'distribution': 'log_uniform', 'min': -6.5, 'max': -5.5},
  'n_epochs': {'distribution': 'int_uniform', 'min': 60, 'max': 100},
  'n_head': {'values': [1, 23]},
  'num_layers': {'distribution': 'int_uniform', 'min': 1, 'max': 3},
  'weight_decay': {'distribution': 'log_uniform', 'min': -6, 'max': -4},
  'noise_level': {'min': 0.01, 'max': 0.05},
  'embed_dim': 8}}

In [43]:
# Data loader
# Noisy Data loader
# Assume that we will take the preprocessing process by the original paper
class CustomDataset(Dataset):
    def __init__(self,config,x_path,y_path):
        super().__init__()
        self.config = config
        self.x_path = x_path
        self.y_path = y_path
        self.load_dataset()
    # Return the length of the data
    def __len__(self):
        return self.data_.shape[0]
    # Defining the dataframe and label for the data before entering the model
    def __getitem__(self,idx):
        return {'input':self.data_[index], 
                'labels': self.labels[index]}

     # The following will be used in order to prepare to use the data_loader to put the dataset into the model
    def load_dataset(self,dataset):
        x_data = {}
        y_data = {}
        x_path_ = self.config['data_dir'] + self.x_path
        y_path_ = self.config['data_dir'] + self.y_path
        
        with h5py.File(x_path_, 'rb') as x_file:
            x_data['data'] = np.array(x_file['data'])
        
        with h5py.File(y_path_, 'rb') as y_file:
            y_data['label'] = np.array(y_file['label'])

        self.data_ = x_data['data'].transpose(0, 2, 1)
        self.data_ = np.expand_dims(self.data_, axis=1)
        
        self.labels = y_data['label']
    #Return the shape of the data_ everytime passing by
    def getshape(self):
        return self.data_.shape[1:]


In [44]:
#Trainer 
class ModelTrainer():
    def __init__(self, model, train_data, criterion, optimizer, device, config):
        self.model = model
        self.train_data = train_data
        self.device = device
        self.config = config
        self.train_loss_list = list()
        self.min_loss = float('inf')
        self.best_model = None
        self.best_optimizer = None
        self.optimizer = optimizer
        self.criterion = criterion

    def train_epoch(self, epoch):
        train_loss = 0.0
        self.model.train()
        for x, rul in self.train_data:
            self.model.zero_grad()
            out = self.model(x.to(self.device).float())
            loss = torch.sqrt(self.criterion(out.float(), rul.to(self.device).float())) # RMSE
            loss.backward()
            self.optimizer.step()
            train_loss += loss

        train_loss = train_loss / len(self.train_data)
        wandb.log({"train loss": train_loss})
        self.train_loss_list.append(train_loss)

        if train_loss < self.min_loss:
            self.min_loss = train_loss
            self.best_model = deepcopy(self.model.state_dict())
            self.best_optimizer = deepcopy(self.optimizer.state_dict())
            self.best_epoch_in_round = epoch
    # Set up the epoch for trainning process
    def train(self):
        self.model.to(self.device)

        for epoch in range(1, self.config['n_epochs'] + 1):
            self.train_epoch(epoch)
            wandb.log({"epoch": epoch})

        self.config['train_loss_list'] = self.train_loss_list

    def update_config(self):
        return self.config




In [46]:
sweep_id = wandb.sweep(sweep=config_wdb, project='RUL_Bearing')

CommError: argument of type 'int' is not iterable

In [47]:
#Train
torch.manual_seed(42)
def training():
    with wandb.init(config = config_wbd):
        config - wandb.config
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        train_data = CustomDataset(config, '/home/quang/Documents/XAI_env-main/data/processed/IMS/x_train.hdf5','/home/quang/Documents/XAI_env-main/data/processed/IMS/y_train.hdf5')
        train_loader = DataLoader(train_data,
                                  batch_size=128,
                                  shuffle=True)
        shape = train_data.getshape()
        model = Net(d_model=config['d_model'],
                    nhead=config['n_head'],
                    feature_size = shape,
                    noise_level = config['noise_level'],
                    embed_dim = config['embed_dim'],
                    n_head = configp['n_head'],
                    num_layers=config['num_layers'],
                    dropout=config['dropout'],
                    l_win=config['l_win'])

        optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], weight_decay=config['weight_decay'])
        criterion = nn.MSELoss()
        trainer = ModelTrainer(model, train_loader, criterion, optimizer, device, config)
        trainer.train()

        test_data = CustomDataset(config,'/home/quang/Documents/XAI_env-main/data/processed/IMS/x_test.hdf5','/home/quang/Documents/XAI_env-main/data/processed/IMS/y_test.hdf5')
        test_loader = DataLoader(test_data,
                                  batch_size=128,
                                  shuffle=True)
        model.to(device)
        test_loss = 0.0
        test_loss_list = list()
        pred_list = list()
        with torch.no_grad():
            for x, rul in test_loader:
                out = model(x.to(device).float())
                loss = torch.sqrt(criterion(out.float(), rul.to(device).float()))
                test_loss += loss
                test_loss_list.append(loss)
                pred_list.append(out.float())

        test_loss_avg = test_loss / len(test_loader)
        config['truth_list'] = truth_list
        config['pred_list'] = pred_list
        config['test_loss_avg'] = test_loss_avg
        config['test_loss_list_per_id'] = test_loss_list
        wandb.log({"test_loss_avg": test_loss_avg})





        vak_data = CustomDataset(config,'/home/quang/Documents/XAI_env-main/data/processed/IMS/x_val.hdf5','/home/quang/Documents/XAI_env-main/data/processed/IMS/y_val.hdf5' )
        val_loader = DataLoader(val_data,
                                 batch_size=128,
                                 shuffle=True)
        model.to(device)
        val_loss = 0.0
        val_loss_list = list()
        with torch.no_grad():
            for x, rul in val_loader:
                out = model(x.to(device).float())
                loss = torch.sqrt(criterion(out.float(), rul.to(device).float()))
                val_loss += loss
                val_loss_list.append(loss)
        val_loss_avg = val_loss / len(test_loader)
        config['val_loss_avg'] = test_loss_avg
        config['val_loss_list_per_id'] = test_loss_list
        wandb.log({"val_loss_avg": test_loss_avg})

In [38]:
!date
wandb.agent(sweep_id, function=trainning ,count = 200)

Thứ tư, 31 Tháng 5 năm 2023 14:03:27 +07


NameError: name 'sweep_id' is not defined